# 0. Importing Relevant Python Packages

First step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following scientific libraries:
<ol type="1">
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. </li>
</ol>
<br>
The first step is to <b>import all required python modules:</b>

In [1]:
import os # for chdir, getcwd, path.basename, path.exists
import copy
import subprocess # for check_call
from osgeo import gdal as GD  
import numpy as np  
#import pandas as pd
import matplotlib.pylab as plt # for add_subplot, cm.jet, colorbar, figure, grid, imshow, rcParams.update, savefig,
                               # set_bad, set_clim, set_title, set_xlabel, set_ylabel
import numpy as np # for arange, arctan, concatenate, cos, fromfile, isnan, ma.masked_value, min, pi, power, reshape,
                   # sqrt, square, sin, sum, tile, transpose, where, zeros     

# 2.Visualizing the InSAR Data

In [2]:
path = f"{os.getcwd()}/data"
if not os.path.exists(path):
    os.makedirs(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /home/x/ictp/obs/javed-coseismic/workplace/tutorials/data


In [ ]:
displacement_map_path = '/mnt/c/Tutorials/data/20190916_20190928.geo.unw.tif'
displacement_map = os.path.basename(displacement_map_path)
data_set = GD.Open(r'20190916_20190928.geo.unw.tif')  

In [ ]:
print(data_set.RasterCount)  

In [ ]:
band_1 = data_set.GetRasterBand(1)
b1 = band_1.ReadAsArray()  

In [ ]:
f = plt.figure()  
plt.imshow(b1)  
plt.savefig('Tiff.png')  
plt.show()  

In [ ]:
!gdalwarp -te 73.6 33 73.9 33.2 20190916_20190928.geo.unw.tif clippedByCoords.tif
!gdalwarp -te 73.6 33 73.9 33.2 20190916_20190928.geo.cc.tif clippedByCoords.cc.tif
!gdalwarp -te 73.6 33 73.9 33.2 100A_05637_131313.geo.E.tif 20190916_20190928.E.tif
!gdalwarp -te 73.6 33 73.9 33.2 100A_05637_131313.geo.U.tif 20190916_20190928.U.tif
!gdalwarp -te 73.6 33 73.9 33.2 100A_05637_131313.geo.N.tif 20190916_20190928.N.tif

In [ ]:
data_set = GD.Open(r'clippedByCoords.tif') 
band_1 = data_set.GetRasterBand(1)
b1 = band_1.ReadAsArray()  
f = plt.figure()  
plt.imshow(b1)  
plt.savefig('Tiff_clip.png')  
plt.show()  

In [ ]:
!gdal_translate -of XYZ clippedByCoords.tif data.xyz

<b>Define some variables:</b>

In [ ]:
sample = 300
samsple=int(sample)
line = 200
line=int(line)
posting_x = 90.0
posting_x=int(posting_x)
posting_y = 90.0
posting_y=int(posting_y)
half_wave = 28.3
half_wave=int(half_wave)

In [ ]:
filename = "data.xyz"
xyz_file=np.genfromtxt(filename, skip_header=0, dtype='unicode')
coordinates = (xyz_file[:,0:2])
los=(xyz_file[:,2:])
coordinates = coordinates.astype(float)
los = los.astype(float)
#print(coordinates)
#print(los)

In [ ]:
#print(los)

In [ ]:
observed_displacement_map = np.reshape(los, (line, sample))

Now we scale the measured and unwrapped InSAR phase into surface displacement in *cm* units and <b>replace all ```nans``` with 0</b>

In [ ]:
observed_displacement_map = observed_displacement_map*half_wave/2.0/np.pi
where_are_NaNs = np.isnan(observed_displacement_map)
observed_displacement_map[where_are_NaNs] = 0

 <b>Create a mask</b> that removes invalid samples (low coherence) from the displacement map:

In [ ]:
observed_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, observed_displacement_map)

In [ ]:
#print(observed_displacement_map_m)

<hr>

## 2.2 Visualize The Surface Displacement Map

We will visualize the displacement map both in units of [cm] and as a rewrapped interferogram.
<br><br>
<b>Write a function that calculates the bounding box.</b>

In [ ]:
def extents(vector_component):
        delta = vector_component[1] - vector_component[0]
        return [vector_component[0] - delta/2, vector_component[-1] + delta/2]

<b>Create a directory in which to store the plots we are about to make, and move into it:</b>

In [ ]:
os.chdir(path)
product_path = 'plots'
if not os.path.exists(product_path):
    os.makedirs(product_path)
if os.path.exists(product_path) and os.getcwd() != f"{path}/{product_path}":
    os.chdir(product_path)
print(f"Current working directory: {os.getcwd()}")

<b>Write a plotting function</b>:

In [ ]:
def plot_model(infile, line, sample, posting, output_filename=None, dpi=72):
    # Calculate the bounding box
    extent_xvec = extents((np.arange(1, sample*posting_x, posting)) / 1000)
    extent_yvec = extents((np.arange(1, line*posting_y, posting)) / 1000)
    extent_xy = extent_xvec + extent_yvec
    
    plt.rcParams.update({'font.size': 14})
    inwrapped = (infile/10 + np.pi) % (2*np.pi) - np.pi
    cmap = copy.copy(plt.cm.get_cmap("jet"))
    cmap.set_bad('white', 1.)
    
    # Plot displacement
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(1, 2, 1)
    im = ax1.imshow(infile, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
    ax1.set_title("Displacement in look direction [mm]")
    ax1.set_xlabel("Easting [km]")
    ax1.set_ylabel("Northing [km]")
    plt.grid()
    
    # Plot interferogram
    im.set_clim(-80, 80)
    ax2 = fig.add_subplot(1, 2, 2)
    im = ax2.imshow(inwrapped, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax2.figure.colorbar(im, ax=ax2, orientation='horizontal')
    ax2.set_title("Interferogram phase [rad]")
    ax2.set_xlabel("Easting [km]")
    ax2.set_ylabel("Northing [km]")
    plt.grid()
    
    if output_filename:
        plt.savefig(output_filename, dpi=dpi)

Call plot_model() to <b>plot our observed displacement map:</b>

In [ ]:
plot_model(observed_displacement_map_m, line, sample, posting_x, output_filename='Okmok-inflation-observation.png', dpi=200)

<hr>

## 3.2 Projecting Mogi Displacement to InSAR Line-of-Sight

In our example, the $x$-axis points east, $y$ points north, and $z$ points up. However, in the code the input values for $z$ are assumed to be depth, such that the Mogi source is at depth $z_s > 0$. The observed interferogram is already corrected for the effect of topography, so the observations can be considered to be at $z = 0$.
    
<img style="padding: 7px" src="https://courses.edx.org/asset-v1:AlaskaX+SAR-401+3T2020+type@asset+block@Lab6-LOSprojection.JPG" width="650" align="center" />
The satellite “sees” a projection of the 3D ground displacement, $u$, onto the look vector, $\hat{L}$, which points from the satellite to the target. Therefore, we are actually interested in the (signed magnitude of the) projection of $u$ onto $\hat{L}$ (right). This is given by

\begin{array}{lcl} proj_{\hat{L}}u & = & (u^T\hat{L})\hat{L} \\ u^T\hat{L} & = & u \cdot \hat{L} = |u||\hat{L}|cos(\alpha) = |u|cos(\alpha) \\ & = & u_x\hat{L}_x+ u_y\hat{L}_y + u_z\hat{L}_z \end{array}

where the look vector is given by $\hat{L}=(sin(l) \cdot cos(t), -sin(l) \cdot sin(t), -cos(l))$, where $l$ is the look angle measured from the nadir direction and $t$ is the satellite track angle measured clockwise from geographic north. All vectors are represented in an east-north-up basis.

Our forward model takes a source, $(x_s,y_s,z_s)$, and computes the look displacement at any given $(x, y, z)$ point. If we represent the <i>i</i>th point on our surface grid by $x_i = (x_i,y_i,z_i)$ the the displacement vector is $u_i = u(x_i, y_i, z_i)$, and the look displacement is

\begin{equation}
d_i = u_i \cdot \hat{L}
\end{equation}



<hr>